# 準備

## 1-1. 作業用ディレクトリ準備

In [ ]:
#@markdown ## 1. 作業用ディレクトリを準備する

# マウント
from google.colab import drive 
drive.mount('/content/drive')

# 作業用ディレクトリを作成して移動
EXP_DPATH = '/content/drive/MyDrive/SFC2021_clip_prefix_caption'
!mkdir -p $EXP_DPATH
%cd $EXP_DPATH

## 1-2. ライブラリのインストール&インポート

In [ ]:
# インストール

import os
import sys

# レポジトリをクローンして移動
REPO_DPATH = os.path.join(EXP_DPATH, 'clip-prefix-caption-jp')
if not os.path.exists(REPO_DPATH):
    !git clone https://github.com/ohashi56225/clip-prefix-caption-jp.git
%cd $REPO_DPATH

# 必要ライブラリインストール
!pip install git+https://github.com/openai/CLIP.git
!pip install scikit-image torch transformers sentencepiece
sys.path.append(REPO_DPATH)

In [1]:
# インポート

import json
import random
from preprocess import prepare_data
from test import Predictor
from model import build_model
from IPython.display import display
from google.colab import files

# ついでに後で使う関数も作っておく
def upload_file():
  uploaded = files.upload()
  if not uploaded:
    image_fpath = ''
  elif len(uploaded) == 1:
    image_fpath = list(uploaded.keys())[0]
  else:
    raise RuntimeError("1度に1枚まで")
  return image_fpath

# 2. cocoデータセットのみで学習

## 2-1. cocoデータの準備
キャプション一覧データと画像データを用意する．各データのフォーマットは，本リポジトリの`README.md`を参照．

In [3]:
# キャプション一覧データのパス
captions_fpath = "data/coco/captions.json"

# 画像データのパス
image_dpath = "data/coco/images"

## 2-2. 学習データの生成
CLIPモデルを用いて，用意したcocoデータから学習データを作成する．

In [ ]:
# train/test/validの割合を指定
coco_test_fpath, coco_valid_fpath, coco_train_fpath = prepare_data(captions_fpath=captions_fpath,
                                                                   image_dpath=image_dpath,
                                                                   test_ratio=0.1,
                                                                   valid_ratio=0.1,
                                                                   train_ratio=0.8,
                                                                   shuffle=False)

## 2-3. 学習

In [4]:
!python train.py --model_name coco \
                 --train_data_fpath $coco_train_fpath \
                 --valid_data_fpath $coco_valid_fpath \
                 --epochs 10 \
                 --batch_size 40

Data size is 105392
Data size is 13174
Train both prefix and GPT
Resume pretrained weights from checkpoints/coco-001.pt
>>> Epoch: 2
coco valid: 100%|██████████████████| 329/329 [00:57<00:00,  5.70it/s, loss=1.66]
>>> Epoch: 3
coco train:  20%|███▍             | 527/2634 [05:00<19:58,  1.76it/s, loss=1.66]

## 2-4. 推論
テスト画像を使用して，学習済みモデルをテストする

In [5]:
# テスト画像リスト
TEST_IMAGE_FNAME_LIST = json.load(open(os.path.splitext(coco_test_fpath)[0]+"_list.json"))

# モデル読込
coco_model = build_model(model_fpath='checkpoints/coco-002.pt')
coco_predictor = Predictor(model=coco_model)

Train both prefix and GPT
Resume pretrained weights from checkpoints/coco-002.pt


In [8]:
# キャプション生成
image_fname = TEST_IMAGE_FNAME_LIST[100]
image_fpath = os.path.join(image_dpath, image_fname)
pil_image, captions = coco_predictor.caption(image_fpath=image_fpath, beam_size=5)
display(pil_image)
print(json.dumps(captions, indent=2, ensure_ascii=False))

# 3. sfcocoデータで追加学習
cocoデータで学習したモデルを，sfcocoデータによって追加学習する．

## 3-1. sfcocoデータを用意

In [10]:
# キャプション一覧データのパス
captions_fpath = "data/sfcoco/captions.json"

# 画像データのパス
image_dpath = "data/sfcoco/images"

## 3-2. 学習用データを生成

In [ ]:
sfcoco_test_fpath, sfcoco_valid_fpath, sfcoco_train_fpath = prepare_data(captions_fpath=captions_fpath,
                                                                         image_dpath=image_dpath,
                                                                         test_ratio=0.1,
                                                                         valid_ratio=0.1,
                                                                         train_ratio=0.8,
                                                                         shuffle=True)

In [1]:
image_dpath = "data/sfcoco/images"
sfcoco_train_fpath = "data/sfcoco/processed/train.pkl"
sfcoco_valid_fpath = "data/sfcoco/processed/valid.pkl"
sfcoco_test_fpath = "data/sfcoco/processed/test.pkl"

## 3-3. 学習

In [ ]:
!python train.py --model_name cocosfcoco \
                 --pretrained_fpath checkpoints/coco-001.pt \
                 --train_data_fpath $sfcoco_train_fpath \
                 --valid_data_fpath $sfcoco_valid_fpath \
                 --epochs 20 \
                 --batch_size 4

In [12]:
from model import build_model
from test import Predictor

In [16]:
# テスト画像リスト
TEST_IMAGE_FNAME_LIST = json.load(open(os.path.splitext(sfcoco_test_fpath)[0]+"_list.json"))

# モデル読込
cocosfcoco_model = build_model(model_fpath='checkpoints/cocosfcoco-008.pt')
cocosfcoco_predictor = Predictor(model=cocosfcoco_model)

Train both prefix and GPT
Resume pretrained weights from checkpoints/cocosfcoco-008.pt


In [ ]:
# キャプション生成
for image_fname in TEST_IMAGE_FNAME_LIST:
    image_fpath = os.path.join(image_dpath, image_fname)
    pil_image, captions = cocosfcoco_predictor.caption(image_fpath=image_fpath, beam_size=5)
    print(captions)